<a href="https://colab.research.google.com/github/Ali-MH-Mansour/Document-classification/blob/main/Document_classification_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Upload SBERT

In [ ]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 391.4 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none

In [ ]:
from sentence_transformers import SentenceTransformer

bert_model = SentenceTransformer('all-mpnet-base-v2')
#multi-qa-mpnet-base-dot-v1
#all-mpnet-base-v2

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import numpy as np
from scipy.spatial.distance import cosine
# Set the maximum length for each document part
max_part_length = 500

def process_documents_sbert(documents):
    """
    Processes a collection of documents and generates embeddings for each part.

    Args:
        documents (list): A list of document strings.

    Returns:
        dict: A dictionary where the keys are the document IDs and the values are document embeddings.
    """
    document_embeddings = {}
    document_embeddings_first = {}
    for doc_id, document in enumerate(documents):
        # Split the document into parts if it's longer than the maximum length
        parts = [document[i:i+max_part_length] for i in range(0, len(document), max_part_length)]
        if len(parts)> 1 and len(parts[-1])<400:
            parts = parts[:-1]
        # Generate embeddings for each part
        part_embeddings = []

        for part in parts:
            vector = bert_model.encode(part)
            part_embeddings.append(vector)

        # Calculate the average of the part embeddings as the document embedding
        document_embedding = np.mean(part_embeddings, axis=0)

        # Store the document embedding
        document_embeddings[doc_id] = document_embedding
        document_embeddings_first[doc_id] = part_embeddings[0] # all

    return document_embeddings, document_embeddings_first

In [ ]:
import pandas as pd

In [ ]:
BBC_file_path="/content/drive/MyDrive/PHD/Data/bbc-text.csv"

data_DF = pd.read_csv(Reuters_file_path)

data_DF.columns.values[0] = "class"
data_DF.columns.values[1] = "document"

data_DF['class'] = pd.factorize(data_DF['class'])[0]
data_DF.head()

,class,document
0,0,BAHIA COCOA REVIEW\n Showers continued throug...
1,1,COMPUTER TERMINAL SYSTEMS &lt;CPML> COMPLETES ...
2,2,N.Z. TRADING BANK DEPOSIT GROWTH RISES SLIGHTL...
3,1,NATIONAL AMUSEMENTS AGAIN UPS VIACOM &lt;VIA> ...
4,3,ROGERS &lt;ROG> SEES 1ST QTR NET UP SIGNIFICAN...


In [ ]:
%%time
docs = data_DF['document'].tolist()
doc2vec_embeddings, embeddings_first  = process_documents_sbert(docs)

CPU times: user 2h 2s, sys: 7.68 s, total: 2h 10s
Wall time: 2h 1min 15s


In [ ]:
# import pickle
# with open("/content/drive/MyDrive/Research/avg_embeddings_re.pkl", "wb") as fOut:
#     pickle.dump(doc2vec_embeddings,fOut)
# with open("/content/drive/MyDrive/Research/first_embeddings_re.pkl", "wb") as fOut:
#     pickle.dump(embeddings_first ,fOut)

In [ ]:
# import pickle
# data_path = "/content/drive/MyDrive/Research/avg_embeddings_re.pkl"

# with open(data_path,'rb') as fin:
#     doc2vec_embeddings = pickle.load(fin)

# data_path2 = "/content/drive/MyDrive/Research/first_embeddings_re.pkl"

# with open(data_path2,'rb') as fin:
#     doc2vec_embeddings2 = pickle.load(fin)



In [ ]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.metrics import f1_score

#-------------------------------------------------------------------------------
classes = data_DF['class'].tolist()
x_train , x_test , y_train , y_test = train_test_split(embeddings_first , classes , test_size = 0.30 , random_state = 0)

kr = ["poly", "rbf","linear"]
for k in kr:

  svm =  SVC(kernel=k, random_state=44).fit(x_train , y_train)
  y_pred=svm.predict(x_test)
  F1 = f1_score(y_test,y_pred, average="micro")
  print("The micro average-weighted f1 score = ", F1, " Kernel ", k)

The micro average-weighted f1 score =  0.8890948409020698  Kernel  poly
The micro average-weighted f1 score =  0.8949644732777263  Kernel  rbf
The micro average-weighted f1 score =  0.8909484090206982  Kernel  linear
CPU times: user 1min 56s, sys: 232 ms, total: 1min 56s
Wall time: 1min 58s


# Split data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

#-------------------------------------------------------------------------------
classes = data_DF['class'].tolist()
x_train , x_test , y_train , y_test = train_test_split(embeddings_first , classes , test_size = 0.30 , random_state = 0)

## Neural net classifier

In [ ]:
%%time
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=500).fit(x_train , y_train)
y_pred=clf.predict(x_test)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.8847698486252703
CPU times: user 1min 30s, sys: 33.2 s, total: 2min 3s
Wall time: 1min 15s


In [ ]:
%%time
# for tfidf change Xvectors, Xtest
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5).fit(x_train , y_train)

y_pred= neigh.predict(x_test)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.8628359592215014
CPU times: user 1.5 s, sys: 21.9 ms, total: 1.52 s
Wall time: 1.51 s


# TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

labels = data_DF['class'].tolist()
x_train , x_test , y_train , y_test = train_test_split(data_DF['document'] , classes , test_size = 0.30 , random_state = 0)
vectorizer = TfidfVectorizer(use_idf=True, max_df = 0.5,min_df = 10, ngram_range=(1,1))

In [ ]:
Xvectors = vectorizer.fit_transform(x_train)

In [ ]:
X_test = vectorizer.transform(x_test)

kr = ["poly","linear", "rbf"]
for k in kr:

  svm =  SVC(kernel=k, random_state=44).fit(Xvectors , y_train)
  y_pred=svm.predict(X_test)
  F1 = f1_score(y_test,y_pred, average="micro")
  print("The micro average-weighted f1 score = ", F1, " Kernel ", k)

The micro average-weighted f1 score =  0.7232004942848317  Kernel  poly
The micro average-weighted f1 score =  0.8934198331788693  Kernel  linear
The micro average-weighted f1 score =  0.8668520234785296  Kernel  rbf


In [ ]:
%%time
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=500).fit(Xvectors , y_train)
y_pred=clf.predict(X_test)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.8974358974358975
CPU times: user 1min 25s, sys: 59.7 s, total: 2min 25s
Wall time: 1min 29s


In [ ]:
%%time
# for tfidf change Xvectors, Xtest
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5).fit(Xvectors , y_train)


y_pred= neigh.predict(X_test)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.8600556070435589
CPU times: user 1min 52s, sys: 94.7 ms, total: 1min 52s
Wall time: 1min 53s


# LSA

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [ ]:
svd = TruncatedSVD(300)
lsa = make_pipeline(svd, Normalizer(copy=False))

In [ ]:
# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(Xvectors)

In [ ]:
X_test_lsa = lsa.transform(X_test)

In [ ]:
kr = ["poly","linear", "rbf"]
for k in kr:

  svm =  SVC(kernel=k, random_state=44).fit(X_train_lsa , y_train)
  y_pred=svm.predict(X_test_lsa)
  F1 = f1_score(y_test,y_pred, average="micro")
  print("The micro average-weighted f1 score = ", F1, " Kernel ", k)

The micro average-weighted f1 score =  0.9730538922155688  Kernel  poly
The micro average-weighted f1 score =  0.9775449101796407  Kernel  linear
The micro average-weighted f1 score =  0.9760479041916168  Kernel  rbf


In [ ]:
%%time
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=500).fit(X_train_lsa , y_train)
y_pred=clf.predict(X_test_lsa)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.9820359281437125
CPU times: user 5.16 s, sys: 2.33 s, total: 7.49 s
Wall time: 3.99 s


In [ ]:
%%time
# for tfidf change Xvectors, Xtest
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5).fit(X_train_lsa , y_train)


y_pred= neigh.predict(X_test_lsa)

F1 = f1_score(y_test,y_pred, average="micro")
print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.9461077844311377
CPU times: user 107 ms, sys: 1.17 ms, total: 108 ms
Wall time: 134 ms


# concat

In [ ]:
# with open("/content/drive/MyDrive/Research/first_embeddings_bbc.pkl", 'rb') as ff:
#   dddd = pickle.load(ff)

In [ ]:
import numpy as np
from scipy.spatial.distance import cosine
# Set the maximum length for each document part
max_part_length = 500

def process_documents_sbert_cncat(documents):
    """
    """
    document_embeddings_concat = {}
    for doc_id, document in enumerate(documents):
        # Split the document into parts if it's longer than the maximum length
        part1 = document[:max_part_length]
        part2 = document[-max_part_length:]

        vector1, vector2 = bert_model.encode([part1, part2])
        # vector2 = bert_model.encode(part2)
        # Calculate the average of the part embeddings as the document embedding

        # Store the document embedding
        concatenated_vector = np.concatenate((vector1, vector2), axis=None)
        document_embeddings_concat[doc_id] = concatenated_vector


    return document_embeddings_concat

In [ ]:
%%time
embds = process_documents_sbert_cncat(docs)

CPU times: user 28min 53s, sys: 2.79 s, total: 28min 56s
Wall time: 29min 8s


In [ ]:
# scroll up and test embds there